# Concept

This notebook contains some common algorithms in ML and DL which are asked in interviews. 

## Linear Regression


In [29]:
# least squares model
class LinearRegressionLS(): 
    def __init__(self): 
        self.slope= None
        self.intercept= None
        
    def forward(self,X): 
        y_pred=[]
        for x in X:
            y_pred.append((self.slope*x) + self.intercept)
        return y_pred
            
    def fit(self,X,y): 
        X = X.float()
        y = y.float()
        x_mean= torch.mean(X,dim=0)
        y_mean= torch.mean(y,dim=0)
        num=torch.sum((X-x_mean)* (y-y_mean))
        den= torch.sum((X-x_mean)**2)
        self.slope= num/den
        self.intercept= y_mean- (self.slope*x_mean)

In [31]:
# least squares test
X=torch.tensor([1,2,3,4,5])
y=torch.tensor([2,4,6,8,10])
lr=LinearRegressionLS()
lr.fit(X,y)
print(lr.slope)
print(lr.intercept)
y_pred=lr.forward(X)
print(y_pred)

tensor(2.)
tensor(0.)
[tensor(2.), tensor(4.), tensor(6.), tensor(8.), tensor(10.)]


In [94]:
# gd method
class LinearRegressionGD():
    def __init__(self,features= 1): 
        self.w= torch.randn(features,1)
        self.b= torch.randn(1)
    def forward(self,X): 
        X = X.float()
        return (X@self.w) + self.b
    def fit(self, X, y, num_epoch=1000, lr=0.01, regul=0.01): 
        X = X.float()
        y = y.float()
        for epoch in range(num_epoch): 
            y_pred= self.forward(X)
            loss= torch.mean((y-y_pred)**2) +regul*torch.sum(self.w**2)
            dw= 2* torch.mean(X.T@ (y_pred-y)) + 2* regul*self.w
            db= 2* torch.mean((y_pred-y)) 
            self.w-= lr*dw
            self.b-= lr*db

# gd test
X=torch.tensor([[1],[2],[3],[4],[5]])
y=torch.tensor([[2],[4],[6],[8],[10]])
lr=LinearRegressionGD()
lr.fit(X,y)
print(lr.w)
print(lr.b)
y_pred=lr.forward(X)
print(y_pred)

tensor([[2.0056]])
tensor([-0.0220])
tensor([[ 1.9837],
        [ 3.9893],
        [ 5.9950],
        [ 8.0006],
        [10.0063]])


## Logistic Regression

Logistic Regression is used to do classification based on regression

In [ ]:
# log reg model
class LogisticRegressionGD(): 
    def __init__(self,features=5): 
        self.w= torch.randn(features, 1)
        self.b= torch.randn(1)
    def sigmoid(self,z): 
        return 1/(1+torch.exp(-z))
    def forward(self,X): 
        z= (X@self.w) + self.b
        y_pred= self.sigmoid(z)
        return y_pred
    def fit(self,X,y,num_epochs=1000, lr=0.01, regul=0.01): 
        y_pred= self.forward(X)
        loss = -torch.mean(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred )) + regul * torch.sum(self.w ** 2)
        dw = torch.mean(X.T @ (y_pred-y)) + 2 * regul * self.w  
        db = torch.mean((y_pred-y))
        self.w -= lr * dw
        self.b -= lr * db


# log reg test
X = torch.tensor([[1, 0, 0, 0, 1], [1, 1, 1, 0, 1]], dtype=torch.float32)
y = torch.tensor([[0], [1]], dtype=torch.float32)

lr = LogisticRegressionGD()
lr.fit(X, y, num_epochs=100, lr=0.01, regul=0.01)
print(lr.w)
print(lr.b)
y_pred=lr.forward(X)
print(y_pred)

tensor([[-0.3114],
        [ 0.8365],
        [ 0.2881],
        [-0.9445],
        [-0.6534]])
tensor([0.0890])
tensor([[0.2941],
        [0.5619]])


In [ ]:
## SVM from scratch

class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)

        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)


In [ ]:
class SVM(): 
    def __init__(self,lr, n_iters,lambdaparam ): 
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None
    def predict(self,X): 
        approx= X@self.w- self.b
        return torch.sign(approx)
    def fit(X,y): 
        ns,nf=X.shape
        y_= np.where(y<=0,-1,1)
        self.w= np.zeros(nf)
        self.b=0
        for r in range(n_iters): 
            for idx,xi in enumerate(X): 
                condition= y_[idx]* ((x_i@self.w)- self.b)>=1
                if condition: 
                    self.w-= self.lr*(2*self.lambdaparam*self.w) 
                else: 
                     self.w-= self.lr*(2*self.lambdaparam*self.w-torch.dot(xi,y_[idx]))
                     self.b -= self.lr * y_[idx]

## Perceptron

In [138]:
# nn.Module
import torch
from torch import nn
class Perceptron(nn.Module): 
    def __init__(self, features=5): 
        super(Perceptron,self).__init__()
        self.w= nn.Parameter(torch.randn(features,1))
        self.b=nn.Parameter(torch.randn(1))
    def sigmoid(self,z): 
        return 1/(1+torch.exp(-z))
    def forward(self,X): 
        return (X@self.w) + self.b
    def fit(self, X, y,lr=0.01,num_epoch=100): 
        loss= nn.CrossEntropyLoss()
        optimizer= torch.optim.SGD(self.parameters(), lr=0.01)
        for i in range(num_epoch): 
            optimizer.zero_grad()
            y_pred=self.forward(X)
            l= loss(y,y_pred)
            l.backward()
            optimizer.step()

X = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5],
                  [0.2, 0.4, 0.6, 0.8, 1.0],
                  [0.3, 0.6, 0.9, 1.2, 1.5],
                  [0.4, 0.8, 1.2, 1.6, 2.0]], dtype=torch.float32)  # shape (4, 5)


y = torch.tensor([[0.2], [0.4], [0.6], [0.8]], dtype=torch.float32)  # shape (4, 1)

# Initialize and train the model
model = Perceptron(features=5)
model.fit(X,y)
print("Predictions:", model.forward(X))

Predictions: tensor([[-0.4500],
        [-1.0072],
        [-1.5644],
        [-2.1217]], grad_fn=<AddBackward0>)


In [127]:
# scratch
import torch
from torch import nn
class Perceptron(): 
    def __init__(self, features= 3): 
        self.w=torch.randn(features,1)
        self.b=(torch.randn(1))
    def sigmoid(self,z): 
        return 1/(1+torch.exp(-z))
    def forward(self, X): 
        z= (X@self.w) + self.b
        y_pred=self.sigmoid(z)
        return y_pred
    def fit(self,X,y,reg_strength=0.01, num_epochs=100, lr=0.1):
        for i in range (num_epochs): 
            y_pred=self.forward(X)
            loss= torch.mean((y-y_pred)**2) + reg_strength*torch.sum(self.w**2)
            dw= 2*torch.mean(X.T@(y_pred-y)) + 2*reg_strength*self.w
            db= 2*torch.mean(y_pred-y)
            self.w-=lr*dw
            self.b-=lr*db
        return self.w,self.b

X = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5],
                  [0.2, 0.4, 0.6, 0.8, 1.0],
                  [0.3, 0.6, 0.9, 1.2, 1.5],
                  [0.4, 0.8, 1.2, 1.6, 2.0]], dtype=torch.float32)  # Shape (4, 5)


y = torch.tensor([[0.2], [0.4], [0.6], [0.8]], dtype=torch.float32)  # Shape (4, 1)

# Initialize and train the model
model = Perceptron(features=5)
w, b = model.fit(X, y)
print("Weights:", w)
print("Bias:", b)
print("Predictions:", model.forward(X))

Weights: tensor([[ 0.8814],
        [-0.0294],
        [ 0.6574],
        [ 0.4297],
        [-0.0130]])
Bias: tensor([-0.9319])
Predictions: tensor([[0.3806],
        [0.4895],
        [0.5993],
        [0.7001]])


## Self Attention

In [18]:
# implementing self attention
import math
import torch
from torch import nn
import torch.nn.functional as F
class SelfAttention(nn.Module): 
    def __init__(self,d,dk,dq,dv): 
        super(SelfAttention, self).__init__()
        self.d= d
        self.dk= dk
        self.dq= dq
        self.dv= dv
        self.wk=nn.Parameter(torch.randn(d,dk))
        self.wq=nn.Parameter(torch.randn(d,dq))
        self.wv= nn.Parameter(torch.randn(d,dv))
    def forward(self,X): 
        K= X@self.wk
        Q= X@self.wq
        V= X@self.wv
        attention_score= (Q@K.T)/math.sqrt(self.dk)
        attention_softmax= F.softmax(attention_score, 
                                     dim=-1)
        attention_output= attention_softmax@V
        return attention_output

In [19]:
# testing self attention
# preparing input
seq= "The lazy brown cat is sleeping"
counts= {w:i for i,w in enumerate(seq.split())}
emb= [counts[k] for k,v in counts.items()]
emb=torch.tensor(emb)
# converting input to embedding
d= 6
vocab_size= 50000
torch.manual_seed(123)
embedder= nn.Embedding(vocab_size,d)
embedding= embedder (emb).detach()
print(embedding)
# apply attention on embedding
attention=SelfAttention(d,dk= 3,dq=3,dv=3)
attention_output= attention(embedding) 
print(attention_output.shape)
print(attention_output)

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603],
        [-0.2196, -0.3792,  0.7671, -1.1925,  0.6984, -1.4097],
        [ 0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205],
        [-0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010],
        [ 0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255],
        [-0.6315, -2.8400, -1.3250,  0.1784, -2.1338,  1.0524]])
torch.Size([6, 3])
tensor([[ 0.2067, -0.7902, -0.5953],
        [-7.2397,  4.2180,  1.3796],
        [-7.1074,  4.1420,  1.3509],
        [ 2.4532, -1.3562, -1.6268],
        [-3.6695,  1.8552,  1.2945],
        [ 4.7616, -0.3846, -0.4100]], grad_fn=<MmBackward0>)


## MLP

In [154]:
import torch 
from torch import nn
class MLPNet(nn.Module): 
    def __init__(self,l,d, dff): 
        super(MLPNet,self).__init__()
        self.l= l
        self.d= d
        self.dff= dff
        self.w1= nn.Parameter(torch.randn(d,dff))
        self.b1= nn.Parameter(torch.randn(l,dff))
        self.w2= nn.Parameter(torch.randn(dff,d))
        self.b2= nn.Parameter(torch.randn(l,d))
    def forward(self,X): 
        l1= torch.matmul(X, self.w1) + self.b1
        activation= torch.relu(l1)
        l2= torch.matmul(activation, self.w2) + self.b2
        return l2
    def fit(self, X,y,num_epochs, lr): 
        loss= nn.CrossEntropyLoss()
        optim= torch.optim.Adam(self.parameters(), lr=lr)
        total_loss= 0
        for i in range (num_epochs): 
            optim.zero_grad()
            y_pred= self. forward(X)
            l= loss(y,y_pred)
            l.backward()
            optim.step()
            lossf = l.item()
        return lossf

In [155]:
l, d, dff = 4, 5, 10  # Example dimensions

# Generate synthetic data
X = torch.randn(l, d)  
y = torch.randn(l, d)  

# Initialize the model
model = MLPNet(l, d, dff)

# Train the model
num_epochs = 100
learning_rate = 0.01
avg_loss = model.fit(X, y, num_epochs=num_epochs, lr=learning_rate)

# Print results
print("Average Loss after Training:", avg_loss)
print("Predictions:", model.forward(X))

Average Loss after Training: -470.34783935546875
Predictions: tensor([[-69.8376, -51.5547, -37.4813, -49.0123, -52.6030],
        [-57.8359, -64.1302, -46.9399, -63.2226, -56.0338],
        [-76.8198, -87.9744, -43.9885, -72.2152, -81.4764],
        [-25.5472, -18.7814, -12.4719, -13.7000, -17.0247]],
       grad_fn=<AddBackward0>)
